目前的协同过滤推荐算法主要分为两大类:1.基于计算相似度的KNN算法。2.基于矩阵分解的SVD算法。先来看看KNN算法
https://blog.csdn.net/weixin_42608414/article/details/87891057

# 使用KNN算法的协同过滤

In [1]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
I2=np.array([[0,5,5,0,0,5,0,3,0,2]])
I1=np.array([[0,4,5,0,4,0,0,0,0,0]])
dist=euclidean_distances(I2,I1)
print('distance between I2 and I1:',dist)

distance between I2 and I1: [[7.41619849]]


加载数据

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
%matplotlib inline
 
movies = pd.read_csv( './data/douban/movies.csv')
print('电影数目（有名称）：%d' % movies[~pd.isnull(movies.title)].shape[0])
print('电影数目（没有名称）：%d' % movies[pd.isnull(movies.title)].shape[0])
print('电影数目（总计）：%d' % movies.shape[0])
movies.sample(10)

电影数目（有名称）：33258
电影数目（没有名称）：24166
电影数目（总计）：57424


,movieId,title
40034,40034,NaN
27576,27576,NaN
12624,12624,The white silk dress
25558,25558,NaN
38956,38956,NaN
19452,19452,The First 9 1/2 Weeks
55998,55998,NaN
17657,17657,毛毛和哈利
50214,50214,NaN
36118,36118,NaN


In [3]:
ratings = pd.read_csv('./data/douban/ratings.csv')
print('用户数据：%d' % ratings.userId.unique().shape[0])
print('电影数据：%d' % ratings.movieId.unique().shape[0])
print('评分数目：%d' % ratings.shape[0])
ratings.head()

用户数据：28718
电影数据：57424
评分数目：2828500


,userId,movieId,rating,timestamp
0,0,0,5,1318222486
1,0,1,4,1313813583
2,0,2,5,1313458035
3,0,3,5,1313327802
4,0,4,3,1312126734


由于评分数据较多,我们将所有的数据都喂给我们的推荐算法，这样会导致内存溢出,会出现"内存错误"的问题，因此我们只要关注公众关注度比较高的电影,也就是那些评价次数多和评分也高的电影,因为那些评价差和评价次数少的电影也没有必要去推荐。

为了找出哪些电影的公众关注度比较高，我们需要整合一下movies表和rating表

In [4]:
combine_movie_rating= pd.merge(ratings,movies,on='movieId')
combine_movie_rating=combine_movie_rating.drop(['timestamp'],axis = 1)
print(len(combine_movie_rating))
combine_movie_rating.head()

2828500


,userId,movieId,rating,title
0,0,0,5,NaN
1,529,0,4,NaN
2,1247,0,5,NaN
3,1335,0,5,NaN
4,1397,0,5,NaN


我们发现有大量的电影名称title为空的记录,所以我们要先过滤掉这些没有电影title的记录

In [5]:
combine_movie_rating = combine_movie_rating.dropna(axis = 0 ,subset=['title'])
print(len(combine_movie_rating))
combine_movie_rating.head()

2604995


,userId,movieId,rating,title
22,0,1,4,Harry Potter and the Deathly Hallows: Part II
23,21,1,4,Harry Potter and the Deathly Hallows: Part II
24,25,1,5,Harry Potter and the Deathly Hallows: Part II
25,34,1,4,Harry Potter and the Deathly Hallows: Part II
26,36,1,5,Harry Potter and the Deathly Hallows: Part II


In [6]:
combine_movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2604995 entries, 22 to 2828497
Data columns (total 4 columns):
userId     int64
movieId    int64
rating     int64
title      object
dtypes: int64(3), object(1)
memory usage: 89.4+ MB


In [7]:
# 删除之后，恢复索引
combine_movie_rating.index=range(combine_movie_rating.shape[0])
combine_movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2604995 entries, 0 to 2604994
Data columns (total 4 columns):
userId     int64
movieId    int64
rating     int64
title      object
dtypes: int64(3), object(1)
memory usage: 69.6+ MB


接下来我要统计一下每部电影总共的评价次数:

In [8]:
movie_rating_count=pd.DataFrame(combine_movie_rating.
                    groupby(['movieId'])['rating'].
                    count().
                    reset_index().
                    rename(columns={'rating':'totalRatingCount'})                   
                   )
movie_rating_count.head()

,movieId,totalRatingCount
0,1,1703
1,2,1080
2,4,1898
3,5,2218
4,10,4981


有了每部电影的总共评价次数以后,我们就可以想办法找出最受关注的电影,最流行的电影了，首先我们先查看一下总评价次数的分布情况


In [9]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_rating_count['totalRatingCount'].describe())

count   33258.000
mean       78.327
std       262.606
min         1.000
25%         3.000
50%        10.000
75%        38.000
max      6574.000
Name: totalRatingCount, dtype: float64


我们可以看到电影总数是33258，其中有50%的电影评价次数小于10次，那说明还有另外50%的电影评价次数高于50%，还记得我上一篇博客中所使用的筛选流行电影的标准就是就是评价次数超过10次就是来源于这里。但是如果一部电影的评价次数只有10次左右的化，那还谈不上是部受关注的电影,所以它不应该被推荐，更不应该被"喂"给推荐算法参加计算。因为推荐算法在执行的时候会消耗大量系统资源,没有价值的数据不应该参与运算,否则会造成系统内存溢出,出现"内存错误"的问题。接下来我们还要查看分位数表中顶层的那10%的数据:

In [10]:
print(movie_rating_count['totalRatingCount'].quantile(np.arange(.9,1,.01)))

0.900    158.000
0.910    184.000
0.920    211.440
0.930    253.000
0.940    303.580
0.950    375.150
0.960    462.000
0.970    590.000
0.980    814.860
0.990   1298.860
Name: totalRatingCount, dtype: float64


我们可以看到有90%的电影评价次数少于158次，那也就是说还有另外10%的电影的评价次数超过了158次。还有9%的电影它们的评价次数超过了184次，还有8%的电影评价次数超过了211次，还有7%的电影评价次数超过了253次。我觉得如果一部电影的评价次数超过了158次的化，那应该是一部受关注的电影了把。那我们就暂时把158次作为识别流行电影的指标吧。(可能你有不同的想法,可以尝试其他指标)，目前电影总数有33258，那10%的话也应该有3325部电影，那我们就决定推荐这3325部电影。

In [11]:
rating_with_totalRatingCount = pd.merge(combine_movie_rating,movie_rating_count,on="movieId")
rating_with_totalRatingCount.sample(5)

,userId,movieId,rating,title,totalRatingCount
652942,3870,751,5,The Matrix,2207
1329093,9476,1925,3,花ざかりの君たちへ：～イケメン♂パラダイス～,470
1268612,1098,1783,3,Per un pugno di dollari,252
2019809,4215,6238,3,炊事班的故事3,101
2163458,6404,8097,5,金瓶梅Ⅱ愛的奴隸,234


In [12]:
(rating_with_totalRatingCount.title=='一一').sum()

1833

In [13]:
#有10%的电影评价次数大于158次
popular_threshold=158
rating_popular_movies= rating_with_totalRatingCount.query('totalRatingCount>=@popular_threshold')
rating_popular_movies.head()

,userId,movieId,rating,title,totalRatingCount
0,0,1,4,Harry Potter and the Deathly Hallows: Part II,1703
1,21,1,4,Harry Potter and the Deathly Hallows: Part II,1703
2,25,1,5,Harry Potter and the Deathly Hallows: Part II,1703
3,34,1,4,Harry Potter and the Deathly Hallows: Part II,1703
4,36,1,5,Harry Potter and the Deathly Hallows: Part II,1703


# 实现KNN算法

我们现在要构造一个用户对电影的评分矩阵,该矩阵每一行代表一个movie，每一列代表一个user，矩阵中的每一个值代表某位用户对某部电影的评分。如果用户对某部电影没有评价那就置为0。然后，我们将矩阵dataframe的值(rating)转换为稀疏矩阵，以便可以进行更有效的计算

In [14]:
from scipy.sparse import csr_matrix
ratings_pivot = rating_popular_movies.pivot(index='movieId', columns='userId',values='rating').fillna(0)
ratings_pivot_sparse = csr_matrix(ratings_pivot.values)

MemoryError: 